## Qestions: **Answer**

In [1]:
import sqlite3
import csv

In [ ]:
def add_poets_data(input_data):
    try:
        # Connecting to database
        conn = sqlite3.connect("poetry_writers_data.db")
        cursor = conn.cursor()
        # Creating table
        cursor.execute("""

        CREATE TABLE IF NOT EXISTS writers (
        author_id PRIMARY KEY,
        poet_name TEXT,
        birth_date Date,
        location TEXT)

        """
        )
        with open(input_data, "r") as csv_file:
            csv_reader = csv.reader(csv_file)
            # skip headers
            next(csv_reader)
            for row in csv_reader:
                cursor.execute("INSERT INTO writers VALUES (?, ?, ?, ?)", row)
        conn.commit()
    except Exception as e:
        print(f"Error: {e}")
    finally:
        conn.close()

# Loading data
data = "/content/poetry_writers.csv"
add_poets_data(data)

In [3]:
# Loading the extension
%load_ext sql

In [4]:
# Using the extension to connect to database
%sql sqlite:///poetry_writers_data.db

In [5]:
# Default to duplicated style
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [6]:
%%sql
SELECT * FROM writers
LIMIT 5;

 * sqlite:///poetry_writers_data.db
Done.


author_id,poet_name,birth_date,location
101,William Wordsworth,1770-04-13,"Grasmere, England"
102,John Keats,None,"Hampstead, England"
103,Emily Dickinson,1830-12-10,"Amherst, Massachusetts"
104,Robert Frost,1874-03-26,"San Francisco, California"
105,"Alfred, Lord Tennyson",1809-08-04,"Somersby, England"


## Question 2: **Answers**

In [7]:
%%sql
CREATE TABLE writers_copy AS
SELECT *
FROM writers;

 * sqlite:///poetry_writers_data.db
(sqlite3.OperationalError) table writers_copy already exists
[SQL: CREATE TABLE writers_copy AS
SELECT *
FROM writers;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


## Questions 3: **Answer**

In [8]:
%%sql
UPDATE writers_copy
SET birth_date = REPLACE(birth_date, 'None', 'Unknown')
WHERE birth_date = 'None';

 * sqlite:///poetry_writers_data.db
0 rows affected.


[]

In [9]:
%%sql
SELECT * FROM writers_copy
LIMIT 5;

 * sqlite:///poetry_writers_data.db
Done.


author_id,poet_name,birth_date,location


## Question 4: **Answer**

In [10]:
%%sql
UPDATE writers_copy
SET birth_date = CASE
                    -- Convert 'DD MM YYYY' to 'YYYY-MM-DD'
                    WHEN birth_date LIKE '__ __ ____'
                THEN SUBSTR(birth_date, 7, 4) || '-' || SUBSTR(birth_date, 4, 2) || '-' ||
                SUBSTR(birth_date, 1, 2)
                    -- Convert 'D MM YYYY' to 'YYYY-MM-DD'
                    WHEN birth_date LIKE '_ __ ____'
                THEN SUBSTR(birth_date, 6, 4) || '-' || SUBSTR(birth_date, 3, 2) || '-0' ||
                SUBSTR(birth_date, 1, 2)
                    -- Handle 'YYYY-MM-DD' (already correct)
                    WHEN birth_date LIKE '____-__-__' THEN birth_date  -- Leave unchanged
                ELSE birth_date
                END
WHERE birth_date  NOT LIKE 'Unknown';

 * sqlite:///poetry_writers_data.db
0 rows affected.


[]

In [11]:
%%sql
UPDATE writers_copy
SET birth_date = DATE(birth_date)
WHERE birth_date  NOT LIKE 'Unknown';

 * sqlite:///poetry_writers_data.db
0 rows affected.


[]

In [12]:
%%sql
SELECT COUNT(*) AS unconverted_columns
FROM writers_copy
WHERE birth_date = 'None'

 * sqlite:///poetry_writers_data.db
Done.


unconverted_columns
0


## Question 5: **Answer**

In [13]:
%%sql
SELECT
    Author_id,
    poet_name,
    CASE
        WHEN birth_date != 'Unknown' THEN SUBSTR(birth_date, 1, 4)
        ELSE 'Unknown'
    END AS birth_year,
    location
FROM writers_copy
LIMIT 5;

 * sqlite:///poetry_writers_data.db
Done.


author_id,poet_name,birth_year,location


## Question 6: **Answer**

In [14]:
%%sql
UPDATE writers_copy
SET poet_name = SUBSTR(poet_name, INSTR(poet_name, ' ') + 1 )
                -- Concatenating the two names with comma and space
                || ', ' ||
                SUBSTR(poet_name, 1, INSTR(poet_name, ' ') - 1 );

 * sqlite:///poetry_writers_data.db
0 rows affected.


[]

In [15]:
%%sql
SELECT
    author_id,
    poet_name
FROM writers_copy
LIMIT 10;

 * sqlite:///poetry_writers_data.db
Done.


author_id,poet_name
